In [1]:
from __future__ import print_function
%matplotlib inline
%pylab inline
# %load_ext memory_profiler
import matplotlib.pyplot as plt
import seaborn; seaborn.set_style('whitegrid')
import numpy
import itertools
from tabulate import tabulate
from pomegranate import *
from pomegranate import BayesianNetwork
import iac_org as iacO

Populating the interactive namespace from numpy and matplotlib


In [2]:
attributes = ["gang", "age", "edu", "mar", "occ", "name"]

def prob(evidences, atts):
    count, total = 0, 0
    for datapoint in iacO.data:
        inc = True
        for key in evidences:
            inc = inc and datapoint[key] == evidences[key]
        if inc:
            total += 1
            f = True
            for key in atts:
                f = f and datapoint[key] == atts[key]
            if f:
                count += 1
    return count, total, float(count)/total

def enums(key):
    vals = set()
    for datapoint in iacO.data:
        if datapoint[key] not in vals:
            vals.add(datapoint[key])
    return list(vals)

In [3]:
import collections
noisy_p_feature_given_hidden = {}
c = 0
for name in enums(attributes[-1]):
    noisy_p_feature_given_hidden[name] = {}
    for att in attributes[:-1]:
        pl = 0.1/(len(enums(att))-1)
        for val in enums(att):
            noisy_p_feature_given_hidden[name][val] = 0.9 if prob({"name":name},{att:val})[2] == 1.0 else pl
            c += 1 if prob({"name":name},{att:val})[2] == 1 else 0
            
print(prob({"name":"George"},{"edu":"JH"})[2], prob({"name":"George"},{"edu":"COL"})[2], prob({"name":"George"},{"edu":"HS"})[2])
print(c/27, noisy_p_feature_given_hidden["George"]["JH"], noisy_p_feature_given_hidden["George"]["COL"], noisy_p_feature_given_hidden["George"]["HS"])

1.0 0.0 0.0
5 0.9 0.05 0.05


In [6]:
distributions = {}
for att in attributes:
    probs = {}
    vals = enums(att)
    for val in vals:
        probs[val] = prob({},{att:val})[2]
    distributions[att] = DiscreteDistribution(probs)

dataInArray = []
n = {}
for datapoint in iacO.data:
    row = []
    for att in attributes:
        row.append(datapoint[att])
    dataInArray.append(row)
    
src = []
dist = []
states = []
for att in attributes:
    src.append(enums(att))
    dist.append(distributions[att])
print(len(dataInArray))
# print(tabulate(dataInArray))

27


In [24]:
# p(h_i|e) = p(h_i)*mul_j(p(e_j|h_i))/sum_k(p(h_k)mul_j(p(e_j|h_k)))

Noisy_CPT = []
d_evidence = {}
for element in itertools.product(*src[:-1]):
    evidences = element
    if evidences not in d_evidence:
        d_evidence[evidences] = {}
    for name in enums("name"):
        if name not in d_evidence[evidences]:
            d_evidence[evidences][name] = 1
        for evi in evidences:
            d_evidence[evidences][name] *= noisy_p_feature_given_hidden[name][evi]
    
eq_evidence = {}
for element in itertools.product(*src[:-1]):
    evidences = element
    eq_evidence[evidences] = 0
    d_evidence[evidences][name] = 27
    for name in enums("name"):
        eq_evidence[evidences] += d_evidence[evidences][name]

c = 0
Noisy_CPT_display = []
Noisy_CPT_dict = {}
for element in itertools.product(*src):
    name = element[-1]
    evidences = element[:-1]
    temp = list(element)
    normalized_noisy_prob = d_evidence[evidences][name]/eq_evidence[evidences]
    Noisy_CPT.append(temp+[normalized_noisy_prob])
    if temp in dataInArray:
        Noisy_CPT_display.append(temp+[normalized_noisy_prob]+[1])
        Noisy_CPT_dict[element] = (normalized_noisy_prob, 1)
        c += 1
    else:
        Noisy_CPT_display.append(temp+[normalized_noisy_prob]+[0])
        Noisy_CPT_dict[element] = (normalized_noisy_prob, 0)

print(tabulate(Noisy_CPT_display, headers=attributes+["noisy prob", "actual"]))

gang    age    edu    mar       occ      name      noisy prob    actual
------  -----  -----  --------  -------  ------  ------------  --------
Jets    20s    HS     single    bookie   Pete     0.0213138           1
Jets    20s    HS     single    bookie   Ned      4.0607e-07          0
Jets    20s    HS     single    bookie   Jim      3.65463e-06         0
Jets    20s    HS     single    bookie   Ralph    3.65463e-06         0
Jets    20s    HS     single    bookie   Nick     7.30927e-06         0
Jets    20s    HS     single    bookie   Clyde    6.57834e-05         0
Jets    20s    HS     single    bookie   Rick     4.0607e-07          0
Jets    20s    HS     single    bookie   John     3.65463e-06         0
Jets    20s    HS     single    bookie   Lance    3.65463e-06         0
Jets    20s    HS     single    bookie   Ken      0.000131567         0
Jets    20s    HS     single    bookie   Mike     6.57834e-05         0
Jets    20s    HS     single    bookie   Fred     0.0011841     

In [36]:
married_status = enums("mar")
names = enums("name")
base_case = ["Jets", "20s", "HS", "single", "bookie", "Pete"]
mSum = 0
nSum = 0
calSum = 0
for stat in married_status:
    new_case = base_case[:3]+[stat]+base_case[4:]
    name = new_case[-1]
    cal = 1
    vals = []
    for val in new_case[:-1]:
        vals.append(noisy_p_feature_given_hidden[name][val])
        cal *= noisy_p_feature_given_hidden[name][val]
    calSum += cal
    print(new_case, Noisy_CPT_dict[tuple(new_case)][1], Noisy_CPT_dict[tuple(new_case)][0], cal, vals)
    mSum += Noisy_CPT_dict[tuple(new_case)][0]

print(mSum, calSum, 0.9*0.9*0.9*0.9)

eSum = 0
for element in itertools.product(*src[:-1]):
    temp = list(element)
    new_case = temp+["Pete"]
    print(new_case, Noisy_CPT_dict[tuple(new_case)][0], Noisy_CPT_dict[tuple(new_case)][1])
    eSum += Noisy_CPT_dict[tuple(new_case)][0]

print(eSum)
    
for name in names:
    new_case = base_case[:-1]+[name]
    print(new_case, Noisy_CPT_dict[tuple(new_case)][1], Noisy_CPT_dict[tuple(new_case)][0])
    nSum += Noisy_CPT_dict[tuple(new_case)][0]

print(nSum)

['Jets', '20s', 'HS', 'single', 'bookie', 'Pete'] 1 0.0213138275121 0.59049 [0.9, 0.9, 0.9, 0.9, 0.9]
['Jets', '20s', 'HS', 'married', 'bookie', 'Pete'] 0 0.00121157930775 0.032805 [0.9, 0.9, 0.9, 0.05, 0.9]
['Jets', '20s', 'HS', 'divorced', 'bookie', 'Pete'] 0 0.00121297298736 0.032805 [0.9, 0.9, 0.9, 0.05, 0.9]
0.0237383798072 0.6561 0.6561
['Jets', '20s', 'HS', 'single', 'bookie', 'Pete'] 0.0213138275121 1
['Jets', '20s', 'HS', 'single', 'burglar', 'Pete'] 0.00120841405946 0
['Jets', '20s', 'HS', 'single', 'pusher', 'Pete'] 0.00118410107433 0
['Jets', '20s', 'HS', 'married', 'bookie', 'Pete'] 0.00121157930775 0
['Jets', '20s', 'HS', 'married', 'burglar', 'Pete'] 6.72121902445e-05 0
['Jets', '20s', 'HS', 'married', 'pusher', 'Pete'] 6.59547877216e-05 0
['Jets', '20s', 'HS', 'divorced', 'bookie', 'Pete'] 0.00121297298736 0
['Jets', '20s', 'HS', 'divorced', 'burglar', 'Pete'] 6.7293430975e-05 0
['Jets', '20s', 'HS', 'divorced', 'pusher', 'Pete'] 6.73059998036e-05 0
['Jets', '20s', 'JH'

In [8]:
cond = ConditionalProbabilityTable(Noisy_CPT, dist[:-1])
name = State(cond, name="name")
gang = State(distributions["gang"], name="gang")
age = State(distributions["age"], name="age")
edu = State(distributions["edu"], name="edu")
mar = State(distributions["mar"], name="mar")
occ = State(distributions["occ"], name="occ")

model = BayesianNetwork("Jets and Sharks")
model.add_states(gang, age, edu, mar, occ, name)

model.add_edge(gang, name)
model.add_edge(age, name)
model.add_edge(edu, name)
model.add_edge(mar, name)
model.add_edge(occ, name)
model.bake()

In [44]:
# print(model.predict_proba({'gang': 'Sharks', 'age': '20s'})) #Content Addressability
# print(model.predict_proba({'gang': 'Sharks', 'age': '30s', 'edu': 'JH', 'occ': 'bookie', 'mar': 'divorced'})) #Graceful Degradation. is actually 'mar': 'single'
# print(model.predict_proba({'name': 'Ike'})) #Retrieval by name
# print(model.predict_proba({'gang': 'Jets'})) #Spontaneous Generalization
print(model.predict_proba({'mar': 'single'})) #Spontaneous Generalization

[{
    "frozen" :false,
    "dtype" :"str",
    "class" :"Distribution",
    "parameters" :[
        {
            "Jets" :0.5555555555555556,
            "Sharks" :0.4444444444444445
        }
    ],
    "name" :"DiscreteDistribution"
}
 {
    "frozen" :false,
    "dtype" :"str",
    "class" :"Distribution",
    "parameters" :[
        {
            "20s" :0.37037037037037013,
            "30s" :0.4814814814814816,
            "40s" :0.14814814814814817
        }
    ],
    "name" :"DiscreteDistribution"
}
 {
    "frozen" :false,
    "dtype" :"str",
    "class" :"Distribution",
    "parameters" :[
        {
            "HS" :0.40740740740740744,
            "JH" :0.3703703703703704,
            "COL" :0.22222222222222227
        }
    ],
    "name" :"DiscreteDistribution"
}
 'single'
 {
    "frozen" :false,
    "dtype" :"str",
    "class" :"Distribution",
    "parameters" :[
        {
            "pusher" :0.33333333333333415,
            "burglar" :0.333333333333333,
            "boo

In [42]:
# print(prob({'gang': 'Jets', 'occ': 'pusher'},{'mar': 'single'}))
# print(model.predict_proba({'occ': 'bookie' ,'mar': 'single'}))
# print(prob({'name': 'Lance', 'occ': 'pusher'},{'mar': 'single'}))
# print(model.predict_proba({'name': 'Lance', 'occ': 'pusher'}))
# print(model.predict_proba({'gang': 'Sharks', 'age': '30s', 'edu': 'JH', 'occ': 'bookie', 'mar': 'married'})) #'mar': 'single'
# print(model.predict_proba({'gang': 'Jets'}))
# print(model.predict_proba({'name': 'John'}))
# print(prob({},{'gang': 'Jets'}))
# print(prob({},{'name': 'John'x}))

[{
    "frozen" :false,
    "dtype" :"str",
    "class" :"Distribution",
    "parameters" :[
        {
            "Jets" :0.08623311650635523,
            "Sharks" :0.9137668834936448
        }
    ],
    "name" :"DiscreteDistribution"
}
 {
    "frozen" :false,
    "dtype" :"str",
    "class" :"Distribution",
    "parameters" :[
        {
            "20s" :0.19633324654467124,
            "30s" :0.7470914357720547,
            "40s" :0.056575317683274055
        }
    ],
    "name" :"DiscreteDistribution"
}
 {
    "frozen" :false,
    "dtype" :"str",
    "class" :"Distribution",
    "parameters" :[
        {
            "HS" :0.03456385399380643,
            "JH" :0.888417057001044,
            "COL" :0.07701908900514963
        }
    ],
    "name" :"DiscreteDistribution"
}
 {
    "frozen" :false,
    "dtype" :"str",
    "class" :"Distribution",
    "parameters" :[
        {
            "single" :0.7724708333239728,
            "married" :0.13035334984189637,
            "divorced" :